In [1]:
import pandas as pd

In [2]:
trips_data_02_df = pd.read_csv('data/201402_trip_data.csv')

In [3]:
trips_data_02_df.shape

(144015, 11)

In [4]:
trips_data_08_df = pd.read_csv('data/201408_trip_data.csv')

In [5]:
trips_data_08_df.shape

(171792, 11)

In [6]:
trips_data_df = trips_data_02_df.append(trips_data_08_df)

In [7]:
trips_data_df.shape

(315807, 12)

In [20]:
trips_data_df.head()

,Bike #,Duration,End Date,End Station,End Terminal,Start Date,Start Station,Start Terminal,Subscriber Type,Subscription Type,Trip ID,Zip Code
0,520,63,8/29/2013 14:14,South Van Ness at Market,66,8/29/2013 14:13,South Van Ness at Market,66,NaN,Subscriber,4576,94127
1,661,70,8/29/2013 14:43,San Jose City Hall,10,8/29/2013 14:42,San Jose City Hall,10,NaN,Subscriber,4607,95138
2,48,71,8/29/2013 10:17,Mountain View City Hall,27,8/29/2013 10:16,Mountain View City Hall,27,NaN,Subscriber,4130,97214
3,26,77,8/29/2013 11:30,San Jose City Hall,10,8/29/2013 11:29,San Jose City Hall,10,NaN,Subscriber,4251,95060
4,319,83,8/29/2013 12:04,Market at 10th,67,8/29/2013 12:02,South Van Ness at Market,66,NaN,Subscriber,4299,94103


In [14]:
end_station_count = trips_data_df['End Station'].value_counts()

In [15]:
start_station_count = trips_data_df['Start Station'].value_counts()

In [16]:
total_count = {}

In [17]:
for entry in end_station_count.iteritems():
    if entry[0] in total_count.keys():
        val_count = total_count[entry[0]]
        new_val_count = val_count + entry[1]
        total_count[entry[0]] = new_val_count
    else:
        total_count[entry[0]] = entry[1]

In [18]:
for entry in start_station_count.iteritems():
    if entry[0] in total_count.keys():
        val_count = total_count[entry[0]]
        new_val_count = val_count + entry[1]
        total_count[entry[0]] = new_val_count
    else:
        total_count[entry[0]] = entry[1]

In [19]:
total_count

{'2nd at Folsom': 12868,
 '2nd at South Park': 17617,
 '2nd at Townsend': 24877,
 '5th at Howard': 12520,
 'Adobe on Almaden': 1374,
 'Arena Green / SAP Center': 1596,
 'Beale at Market': 13193,
 'Broadway St at Battery St': 4833,
 'Broadway at Main': 148,
 'California Ave Caltrain Station': 1268,
 'Castro Street and El Camino Real': 1504,
 'Civic Center BART (7th at Market)': 12967,
 'Clay at Battery': 9822,
 'Commercial at Montgomery': 11352,
 'Cowper at University': 1432,
 'Davis at Jackson': 10150,
 'Embarcadero at Bryant': 13785,
 'Embarcadero at Folsom': 12761,
 'Embarcadero at Sansome': 29286,
 'Embarcadero at Vallejo': 11722,
 'Evelyn Park and Ride': 1334,
 'Franklin at Maple': 297,
 'Golden Gate at Polk': 6830,
 'Grant Avenue at Columbus Avenue': 12858,
 'Harry Bridges Plaza (Ferry Building)': 31062,
 'Howard at 2nd': 13237,
 'Japantown': 2249,
 'MLK Library': 2483,
 'Market at 10th': 16031,
 'Market at 4th': 20501,
 'Market at Sansome': 26858,
 'Mechanics Plaza (Market at Bat

In [21]:
trips_data_df.Duration.describe()

count    315807.000000
mean       1177.384960
std        6350.087274
min          60.000000
25%         347.000000
50%         523.000000
75%         777.000000
max      722236.000000
Name: Duration, dtype: float64